# Imports and helper functions

In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn import metrics
from sklearn.model_selection import ParameterGrid

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#!git clone --recurse-submodules https://github.com/SamiHaddouti/Hybrid-VARMA-Transformer-for-TSF
!git clone https://github.com/SamiHaddouti/Time-Series-Library.git
!git clone https://github.com/SamiHaddouti/PatchTST.git

In [7]:
'''
The following code was derived from here: https://michael-fuchs-python.netlify.app/2020/10/29/time-series-analysis-regression-extension-techniques-for-forecasting-multivariate-variables/ and https://github.com/Apress/hands-on-time-series-analylsis-python/blob/master/Chapter%204/8.%20VARMA%20with%20Gird%20Search.ipynb 
The code provided by https://github.com/Apress/hands-on-time-series-analylsis-python is licensed under the Freeware License, some rights reserved, Copyright (c) 2020 B V Vishwas and Ashish Patel.
Please take a look at the LICENSE file for more information: https://github.com/Apress/hands-on-time-series-analylsis-python/blob/master/LICENSE.txt
'''
def Augmented_Dickey_Fuller_Test_func(timeseries , column_name):
    '''
    Calculates statistical values whether the available data are stationary or not 
    
    Args:
        series (float64): Values of the column for which stationarity is to be checked, numpy array of floats 
        column_name (str): Name of the column for which stationarity is to be checked
    
    Returns:
        p-value that indicates whether the data are stationary or not
    ''' 
    print (f'Results of Dickey-Fuller Test for column: {column_name}')
    adfTest = adfuller(timeseries, autolag='AIC')
    dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
    for key, value in adfTest[4].items():
       dfResults['Critical Value (%s)'%key] = value
    print (dfResults)
    if adfTest[1] <= 0.05:
        print()
        print("Conclusion:")
        print("Reject the null hypothesis")
        print('\033[92m' + "Data is stationary" + '\033[0m')
    else:
        print()
        print("Conclusion:")
        print("Fail to reject the null hypothesis")
        print('\033[91m' + "Data is non-stationary" + '\033[0m')

def cointegration_test_func(df): 
    '''
    Test if there is a long-run relationship between features
    
    Args:
        dataframe (float64): Values of the columns to be checked, numpy array of floats 
    
    Returns:
        True or False whether a variable has a long-run relationship between other features
    ''' 
    johansen_cointegration_test = coint_johansen(df,-1,5)
    c = {'0.90':0, '0.95':1, '0.99':2}
    traces = johansen_cointegration_test.lr1
    cvts = johansen_cointegration_test.cvt[:, c[str(1-0.05)]]
    def adjust(val, length= 6): 
        return str(val).ljust(length)
    print('Column_Name  >  Test_Stat  >  C(95%)  =>  Signif  \n', '--'*25)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), '  >  ', 
              adjust(round(trace,2), 9), "  >  ", 
              adjust(cvt, 8), '  => ' , 
              trace > cvt)

## Load data

In [9]:
# Read ILI dataset and load into pd.Dataframe
illness_df = pd.read_csv("/content/Time-Series-Library/dataset/illness/national_illness.csv")
# Change date colum format to datetime.datetime
illness_df['date'] =  pd.to_datetime(illness_df['date'])

# Read ILI dataset and load into pd.Dataframe
exchange_df = pd.read_csv("/content/Time-Series-Library/dataset/exchange_rate/exchange_rate.csv")
# Change date colum format to datetime.datetime
exchange_df['date'] =  pd.to_datetime(exchange_df['date'])

In [ ]:
illness_df

In [ ]:
exchange_df

# EDA

In [ ]:
print("Illness")
print(f"Length: {len(illness_df)}")
print(f"Columns: {len(illness_df.columns)}")
print(f"Frequency: weekly")
print(f"Timespan: {(pd.Timedelta(illness_df['date'][len(illness_df)-1] - illness_df['date'][0]).days)/7} weeks")

print("\n")

print("Exchange")
print(f"Length: {len(exchange_df)}")
print(f"Columns: {len(exchange_df.columns)}")
print(f"Frequency: daily")
print(f"Timespan: {pd.Timedelta(exchange_df['date'][len(exchange_df)-1] - exchange_df['date'][0]).days} days")

In [13]:
# Set date as index
illness_df.set_index('date', inplace=True)
exchange_df.set_index('date', inplace=True)

In [ ]:
'''
Parts of the following code were derived and adapted from here: https://michael-fuchs-python.netlify.app/2020/10/29/time-series-analysis-regression-extension-techniques-for-forecasting-multivariate-variables/ and https://github.com/Apress/hands-on-time-series-analylsis-python/blob/master/Chapter%204/8.%20VARMA%20with%20Gird%20Search.ipynb 
The code provided by https://github.com/Apress/hands-on-time-series-analylsis-python is licensed under the Freeware License, some rights reserved, Copyright (c) 2020 B V Vishwas and Ashish Patel.
Please take a look at the LICENSE file for more information: https://github.com/Apress/hands-on-time-series-analylsis-python/blob/master/LICENSE.txt
'''
# Plots of ILI variables
for col in illness_df.columns:
  illness_df[col].plot(figsize=(15, 6))
  plt.title(col)
  plt.xlabel("Date")
  plt.ylabel(col)
  plt.show()

In [ ]:
'''
Parts of the following code were derived and adapted from here: https://michael-fuchs-python.netlify.app/2020/10/29/time-series-analysis-regression-extension-techniques-for-forecasting-multivariate-variables/ and https://github.com/Apress/hands-on-time-series-analylsis-python/blob/master/Chapter%204/8.%20VARMA%20with%20Gird%20Search.ipynb 
The code provided by https://github.com/Apress/hands-on-time-series-analylsis-python is licensed under the Freeware License, some rights reserved, Copyright (c) 2020 B V Vishwas and Ashish Patel.
Please take a look at the LICENSE file for more information: https://github.com/Apress/hands-on-time-series-analylsis-python/blob/master/LICENSE.txt
'''
# Plots of Exchange variables
for col in exchange_df.columns:
  exchange_df[col].plot(figsize=(15, 6))
  plt.title(col)
  plt.xlabel("Date")
  plt.ylabel(col)
  plt.show()

# Data pre-processing

## Stationarity check

In [ ]:
'''
Parts of the following code were derived and adapted from here: https://michael-fuchs-python.netlify.app/2020/10/29/time-series-analysis-regression-extension-techniques-for-forecasting-multivariate-variables/ and https://github.com/Apress/hands-on-time-series-analylsis-python/blob/master/Chapter%204/8.%20VARMA%20with%20Gird%20Search.ipynb 
The code provided by https://github.com/Apress/hands-on-time-series-analylsis-python is licensed under the Freeware License, some rights reserved, Copyright (c) 2020 B V Vishwas and Ashish Patel.
Please take a look at the LICENSE file for more information: https://github.com/Apress/hands-on-time-series-analylsis-python/blob/master/LICENSE.txt
'''
# Illness data
for name, column in illness_df[['% WEIGHTED ILI', '%UNWEIGHTED ILI', 'AGE 0-4', 'AGE 5-24', 'ILITOTAL', 'NUM. OF PROVIDERS', 'OT']].iteritems():
    Augmented_Dickey_Fuller_Test_func(illness_df[name],name)
    print('\n')

In [ ]:
'''
Parts of the following code were derived and adapted from here: https://michael-fuchs-python.netlify.app/2020/10/29/time-series-analysis-regression-extension-techniques-for-forecasting-multivariate-variables/ and https://github.com/Apress/hands-on-time-series-analylsis-python/blob/master/Chapter%204/8.%20VARMA%20with%20Gird%20Search.ipynb 
The code provided by https://github.com/Apress/hands-on-time-series-analylsis-python is licensed under the Freeware License, some rights reserved, Copyright (c) 2020 B V Vishwas and Ashish Patel.
Please take a look at the LICENSE file for more information: https://github.com/Apress/hands-on-time-series-analylsis-python/blob/master/LICENSE.txt
'''
# Exchange data
for name, column in exchange_df[['0', '1', '2', '3', '4', '5', '6', 'OT']].iteritems():
    Augmented_Dickey_Fuller_Test_func(exchange_df[name],name)
    print('\n')

### Stationarize data

In [ ]:
# diff_illness_df = illness_df.diff()
# diff_exchange_df = exchange_df.diff()

## Cointegration function

In [19]:
# Illness data
cointegration_test_func(illness_df)

Column_Name  >  Test_Stat  >  C(95%)  =>  Signif  
 --------------------------------------------------
% WEIGHTED ILI   >   458.26      >   111.7797   =>  True
%UNWEIGHTED ILI   >   325.58      >   83.9383    =>  True
AGE 0-4   >   205.26      >   60.0627    =>  True
AGE 5-24   >   125.75      >   40.1749    =>  True
ILITOTAL   >   50.57       >   24.2761    =>  True
NUM. OF PROVIDERS   >   11.38       >   12.3212    =>  False
OT       >   0.13        >   4.1296     =>  False


In [ ]:
# Exchange data
cointegration_test_func(exchange_df)

# Hybrid model


## VARMA

### Grid Search

In [ ]:
'''
Parts of the following code were derived and adapted from here: https://michael-fuchs-python.netlify.app/2020/10/29/time-series-analysis-regression-extension-techniques-for-forecasting-multivariate-variables/ and https://github.com/Apress/hands-on-time-series-analylsis-python/blob/master/Chapter%204/8.%20VARMA%20with%20Gird%20Search.ipynb 
The code provided by https://github.com/Apress/hands-on-time-series-analylsis-python is licensed under the Freeware License, some rights reserved, Copyright (c) 2020 B V Vishwas and Ashish Patel.
Please take a look at the LICENSE file for more information: https://github.com/Apress/hands-on-time-series-analylsis-python/blob/master/LICENSE.txt
'''
# Define param space
param_grid = {'p': [1,2,3], 'q':[1,2,3]}
pq = list(ParameterGrid(param_grid))

def find_optimal_params(train, test):
  '''
    Grid search function that iterates params cofnigurations and calculates the loss of each VARMA model.
    
    Args:
        train (pd.Dataframe): Training data for the model
        test (pd.Dataframe): Test data for the model
    
    Returns:
        (p, q), a list of the optimal params

  ''' 
  df_results = pd.DataFrame(columns=['p', 'q','mae','mse'])
  print('Starting grid search...')
  for i, pq_dict in enumerate(pq):
      # Get params and fit model
      p = pq_dict.get('p')
      q = pq_dict.get('q')
      fitted_model = VARMAX(train, order=(p,q)).fit()
      forecast_test = fitted_model.forecast(len(test))
   # End of the code adapted by https://michael-fuchs-python.netlify.app/2020/10/29/time-series-analysis-regression-extension-techniques-for-forecasting-multivariate-variables/ and https://github.com/Apress/hands-on-time-series-analylsis-python
      # Calculate loss
      mae = mean_absolute_error(test, forecast_test)
      df_results = df_results.append({'p': p, 'q': q, 'mae': mae}, ignore_index=True)

  return list(df_results.sort_values('mae', ascending=True)[['p', 'q']].iloc[0])

### Training and test function

In [ ]:
def train_test_varma(Xy, data_name):
  '''
    Function to train and test VARMA models for a number of prediction lengths.
    
    Args:
        Xy (pd.Dataframe): Training and test data for the model
        data_name (str): Name of dataset used
    
    Returns:
        summary_df, a pd.DataFrame that summarizes the metrics of all VARMA runs
        result_df, a pd.DataFrame that adds the forecast and residuals to the original dataframe
  ''' 
  print(f"Starting training...")
  summary_df = pd.DataFrame(columns=['Run','MAE', 'MSE'])
  results_df = Xy.copy()

  if  data_name=="ILI":
     pred_lens = [24, 36, 48, 60]
  else:
     pred_lens = [96, 192, 336, 720]

  for pred_len in pred_lens:

    # Split into Train and Test data based on pred len
    X, y = Xy[0:-pred_len], Xy[-pred_len:]
    print(f"Fitting model (pred_len={pred_len})...")

    # Find optimal parameters and fit model
    optimal_order = find_optimal_params(X, y)
    print(f"Found optimal params={optimal_order})...")
    var_model = VARMAX(X, order=(optimal_order[0], optimal_order[1]))
    fitted_model = var_model.fit()

    print(f"Testing model (pred_len={pred_len})...")
    # Forecast model
    forecast_test = fitted_model.forecast(len(y))
    results_df['forecast_' + str(pred_len)] = [None]*len(X) + list(forecast_test['OT'])
    results_df['residuals_' + str(pred_len)] = pd.concat([fitted_model.resid['OT'], y['OT'] - forecast_test['OT']], axis = 0)

    # Calculate metrics
    mae = mean_absolute_error(y['OT'], forecast_test['OT'])
    mse = mean_squared_error(y['OT'], forecast_test['OT'])
    summary_df = summary_df.append({'Run': f'VARMA_{data_name}_{pred_len}', 'MAE': mae, "MSE": mse}, ignore_index=True)

    summary = f'VARMA:\n data={data_name} \n pred_len={pred_len} \n order={optimal_order} \n mae={mae} \n mse={mse}'

    #with open(f"summary_VARMA_raw_{data_name}_{pred_len}.txt", "w") as text_file:
     # text_file.write(summary)

    print(summary)

  #results_df.to_csv(f"results_VARMA_raw_{data_name}.csv", index=False)

  print("Training done...")

  return summary_df, results_df

### Run VARMA models for ILI and Exchange data

In [ ]:
ILI_summary_df, ILI_results_df = train_test_varma(illness_df, "ILI")
Exchange_summary_df, Exchange_results_df = train_test_varma(exchange_df, "Exchange")

### Save residuals

In [ ]:
# Add best residuals to ILI data
ILI_results_df['date'] = illness_df.index
ILI_residuals_df = ILI_results_df[['date', '% WEIGHTED ILI', '%UNWEIGHTED ILI', 'AGE 0-4', 'AGE 5-24', 'ILITOTAL', 'NUM. OF PROVIDERS', 'residuals_24', 'OT']].rename(columns={'residuals_24': 'RESIDUALS'})

# Save new ILI dataframe
ILI_residuals_df.to_csv("national_illness_VARMA_Residuals.csv", index=False)

# Add best residuals to Exchange data
Exchange_results_df['date'] = exchange_df.index
Exchange_residuals_df = Exchange_results_df[['date', '0', '1', '2', '3', '4', '5', '6', 'residuals_192', 'OT']].rename(columns={'residuals_192': 'RESIDUALS'})

# Save new Exchange dataframe
Exchange_residuals_df.to_csv("exchange_rate_VARMA_Residuals.csv", index=False)

## Transformer

In [ ]:
%cd /content/Time-Series-Library

In [ ]:
%cd /content/PatchTST/PatchTST_supervised

In [ ]:
!git stash

In [ ]:
!git pull

In [ ]:
!git status

### ILI data

In [ ]:
%cd /content/Time-Series-Library

In [ ]:
!pip install -r requirements.txt

In [ ]:
# Autoformer - Illness
print(f'\nRunning Model with pred_len 24...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_24' --model 'Autoformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 24 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 36...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_36' --model 'Autoformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 36 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 48...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_48' --model 'Autoformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 48 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 60...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_60' --model 'Autoformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 60 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# Autoformer - Illness + VARMA Residuals
print(f'\nRunning Model with pred_len 24...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_24' --model 'Autoformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 24 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 36...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_36' --model 'Autoformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 36 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 48...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_48' --model 'Autoformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 48 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 60...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_60' --model 'Autoformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 60 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# FEDformer - Illness
print(f'\nRunning Model with pred_len 24...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_24' --model 'FEDformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 24 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 36...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_36' --model 'FEDformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 36 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 48...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_48' --model 'FEDformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 48 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 60...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_60' --model 'FEDformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 60 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# FEDformer - Illness + VARMA Residuals
print(f'\nRunning Model with pred_len 24...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_24' --model 'FEDformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 24 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 36...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_36' --model 'FEDformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 36 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 48...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_48' --model 'FEDformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 48 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 60...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_60' --model 'FEDformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 60 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# Pyraformer - Illness
print(f'\nRunning Model with pred_len 24...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_24' --model 'Pyraformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 24 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 36...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_36' --model 'Pyraformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 36 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 48...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_48' --model 'Pyraformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 48 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 60...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_60' --model 'Pyraformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 60 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# Pyraformer - Illness + VARMA Residuals
print(f'\nRunning Model with pred_len 24...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_24' --model 'Pyraformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 24 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 36...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_36' --model 'Pyraformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 36 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 48...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_48' --model 'Pyraformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 48 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 60...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_60' --model 'Pyraformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 60 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# Crossformer - Illness
print(f'\nRunning Model with pred_len 24...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_24' --model 'Crossformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 24 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 36...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_36' --model 'Crossformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 36 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 48...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_48' --model 'Crossformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 48 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 60...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_60' --model 'Crossformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 60 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 7 --dec_in 7 --c_out 7 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# Crossformer - Illness + VARMA Residuals
print(f'\nRunning Model with pred_len 24...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_24' --model 'Crossformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 24 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 36...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_36' --model 'Crossformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 36 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 48...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_48' --model 'Crossformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 48 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 60...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'ili_36_60' --model 'Crossformer' --data 'custom' --root_path './dataset/illness/' --data_path 'national_illness_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 36 --label_len 18 --pred_len 60 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
%cd /content/PatchTST/PatchTST_supervised

In [ ]:
# PatchTST - Illness
!sh /content/PatchTST/PatchTST_supervised/scripts/PatchTST/illness.sh

In [ ]:
# PatchTST - Illness + VARMA Residuals
!sh /content/PatchTST/PatchTST_supervised/scripts/PatchTST/illness_residuals.sh

### Exchange data

In [ ]:
%cd /content/Time-Series-Library

In [ ]:
# Autoformer - Exchange
print(f'\nRunning Model with pred_len 96...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_96' --model 'Autoformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 96 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 192...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_192' --model 'Autoformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 192 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 336...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_336' --model 'Autoformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 336 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 720...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_720' --model 'Autoformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 720 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# Autoformer - Exchange + VARMA Residuals
print(f'\nRunning Model with pred_len 96...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_96' --model 'Autoformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 96 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 192...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_192' --model 'Autoformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 192 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 336...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_336' --model 'Autoformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 336 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 720...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_720' --model 'Autoformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 720 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# FEDformer - Exchange
print(f'\nRunning Model with pred_len 96...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_96' --model 'FEDformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 96 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 192...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_192' --model 'FEDformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 192 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 336...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_336' --model 'FEDformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 336 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 720...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_720' --model 'FEDformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 720 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# FEDformer - Exchange + VARMA Residuals
print(f'\nRunning Model with pred_len 96...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_96' --model 'FEDformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 96 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 192...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_192' --model 'FEDformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 192 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 336...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_336' --model 'FEDformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 336 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 720...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_720' --model 'FEDformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 720 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# Pyraformer - Exchange
print(f'\nRunning Model with pred_len 96...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_96' --model 'Pyraformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 96 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 192...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_192' --model 'Pyraformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 192 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 336...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_336' --model 'Pyraformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 336 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 720...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_720' --model 'Pyraformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 720 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# Pyraformer - Exchange + VARMA Residuals
print(f'\nRunning Model with pred_len 96...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_96' --model 'Pyraformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 96 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 192...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_192' --model 'Pyraformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 192 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 336...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_336' --model 'Pyraformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 336 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 720...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_720' --model 'Pyraformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 720 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# Crossformer - Exchange
print(f'\nRunning Model with pred_len 96...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_96' --model 'Crossformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 96 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 192...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_192' --model 'Crossformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 192 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 336...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_336' --model 'Crossformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 336 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 720...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_720' --model 'Crossformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 720 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 8 --dec_in 8 --c_out 8 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
# Crossformer - Exchange + VARMA Residuals
print(f'\nRunning Model with pred_len 96...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_96' --model 'Crossformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 96 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 192...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_192' --model 'Crossformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 192 --seasonal_patterns  'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 336...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_336' --model 'Crossformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 336 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2
print(f'\nRunning Model with pred_len 720...')
!python run.py --task_name 'long_term_forecast' --is_training 1 --model_id 'Exchange_96_720' --model 'Crossformer' --data 'custom' --root_path './dataset/exchange_rate/' --data_path 'exchange_rate_VARMA_Residuals.csv' --features 'M' --target 'OT' --freq 'h' --checkpoints './checkpoints/' --seq_len 96 --label_len 48 --pred_len 720 --seasonal_patterns 'Monthly' --mask_rate 0.25 --anomaly_ratio 0.25 --top_k 5 --num_kernels 6 --enc_in 9 --dec_in 9 --c_out 9 --d_model 512 --n_heads 8 --e_layers 2 --d_layers 1 --d_ff 2048 --moving_avg 25 --factor 3 --dropout 0.1 --embed 'timeF' --activation 'gelu' --num_workers 10 --itr 1 --train_epochs 10 --batch_size 32 --patience 3 --learning_rate 0.0001 --des 'Exp' --loss 'MSE' --lradj 'type1' --devices '0,1,2,3' --p_hidden_layers 2


In [ ]:
%cd /content/PatchTST/PatchTST_supervised

In [ ]:
# PatchTST - Exchange
!sh /content/PatchTST/PatchTST_supervised/scripts/PatchTST/exchange.sh

In [ ]:
# PatchTST - Exchange + VARMA Residuals
!sh /content/PatchTST/PatchTST_supervised/scripts/PatchTST/exchange_residuals.sh